<a href="https://colab.research.google.com/github/Data-Analytics-with-Python/database-exercise-bastienm69/blob/main/K723_ASSIGNMENT_ONE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import os
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from sklearn.preprocessing import StandardScaler
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import adjusted_rand_score

# ============ USER CONFIGURATION ============
DATA_PATH = "east_west_airlines.csv"
OUTPUT_DIR = "./outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

SAMPLE_FOR_HIERARCHY = True   # If True, dendrograms & elbow use a sample for speed
DENDRO_SAMPLE_N = 200
SCATTER_SAMPLE_N = 2000

CHOSEN_K = 4                 # initial chosen number of clusters
ELBOW_K_MAX = 8
RANDOM_STATE = 40
# ===========================================

# 1. Read dataset and summary statistics
df = pd.read_csv(DATA_PATH)
print(f"Loaded data: {df.shape[0]} rows, {df.shape[1]} columns")
summary = df.describe(include='all').transpose()
summary.to_csv(os.path.join(OUTPUT_DIR, "summary_statistics.csv"))
display(summary.head(20))

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns:", num_cols)

# 2. Develop boxplots for continuous variables
box_dir = os.path.join(OUTPUT_DIR, "boxplots")
os.makedirs(box_dir, exist_ok=True)
for col in num_cols:
    plt.figure(figsize=(6,3))
    plt.boxplot(df[col].dropna())
    plt.title(f"Boxplot: {col}")
    plt.tight_layout()
    plt.savefig(os.path.join(box_dir, f"{col}_boxplot.png"))
    plt.close()

print(f"Saved boxplots to {box_dir}")

# 3. Create scatter plots for all pairs of numerical variables
scatter_dir = os.path.join(OUTPUT_DIR, "scatter_pairs")
os.makedirs(scatter_dir, exist_ok=True)
if SCATTER_SAMPLE_N < len(df):
    scatter_df = df.loc[np.random.RandomState(RANDOM_STATE).choice(df.index, size=SCATTER_SAMPLE_N, replace=False), num_cols]
else:
    scatter_df = df[num_cols]

pairs = []
for i in range(len(num_cols)):
    for j in range(i+1, len(num_cols)):
        x = num_cols[i]; y = num_cols[j]
        plt.figure(figsize=(4,4))
        plt.scatter(scatter_df[x], scatter_df[y], s=8)
        plt.xlabel(x); plt.ylabel(y)
        plt.title(f"{x} vs {y}")
        plt.tight_layout()
        fname = os.path.join(scatter_dir, f"{x}_vs_{y}.png")
        plt.savefig(fname)
        plt.close()
        pairs.append(fname)
print(f"Saved scatter pair plots to {scatter_dir} (sample size = {len(scatter_df)})")

# 4. Apply feature scaling and explanation
scaler = StandardScaler()
X_num = df[num_cols].fillna(0).values
X_scaled = scaler.fit_transform(X_num)
df_scaled = pd.DataFrame(X_scaled, columns=num_cols)
scaling_explanation = (
    "Standard scaling (zero mean, unit variance) is essential for distance-based methods "
    "so variables with larger numeric ranges don't dominate distances."
)
print(scaling_explanation)

# 5. Compute normalized distances
if SAMPLE_FOR_HIERARCHY and len(df) > DENDRO_SAMPLE_N:
    idx = np.random.RandomState(RANDOM_STATE).choice(df.index, size=DENDRO_SAMPLE_N, replace=False)
    sample_scaled = df_scaled.loc[idx].reset_index(drop=True)
    print(f"Using sample of {len(sample_scaled)} for hierarchical clustering/dendrograms.")
else:
    sample_scaled = df_scaled.copy()
    idx = sample_scaled.index

# 6. Apply hierarchical clustering (linkages)
linkages = ['ward', 'centroid', 'average']
Zs = {}
for lk in linkages:
    # ward requires Euclidean; centroid/average ok with euclidean
    Zs[lk] = linkage(sample_scaled.values, method=lk, metric='euclidean')

# 7. Dendrograms for each linkage (save images)
dend_dir = os.path.join(OUTPUT_DIR, "dendrograms")
os.makedirs(dend_dir, exist_ok=True)
for lk, Z in Zs.items():
    plt.figure(figsize=(10,4))
    dendrogram(Z, no_labels=True)
    plt.title(f"Dendrogram ({lk})")
    plt.tight_layout()
    plt.savefig(os.path.join(dend_dir, f"dendrogram_{lk}.png"))
    plt.close()
print("Saved dendrograms:", dend_dir)

# 8. Analyze clusters and their centroids (use CHOSEN_K)
cluster_summaries = {}
for lk, Z in Zs.items():
    labels = fcluster(Z, t=CHOSEN_K, criterion='maxclust')
    # map these back to actual df rows (only for sampled indices)
    sample_df_for_analysis = df.loc[idx].copy().reset_index(drop=True)
    sample_df_for_analysis[f'cluster_h_{lk}'] = labels
    centroids = sample_df_for_analysis.groupby(f'cluster_h_{lk}')[num_cols].mean()
    summary_stats = sample_df_for_analysis.groupby(f'cluster_h_{lk}')[num_cols].agg(['mean','std','count'])
    cluster_summaries[lk] = {"centroids": centroids, "summary": summary_stats}
    # save
    centroids.to_csv(os.path.join(OUTPUT_DIR, f"centroids_hier_{lk}.csv"))
    summary_stats.to_csv(os.path.join(OUTPUT_DIR, f"cluster_summary_hier_{lk}.csv"))

# Parallel coordinates (for a manageable set of features)
pc_dir = os.path.join(OUTPUT_DIR, "parallel_coords")
os.makedirs(pc_dir, exist_ok=True)
pc_cols = num_cols[:8] if len(num_cols) > 8 else num_cols
for lk in linkages:
    sample_df_for_plot = df.loc[idx].copy().reset_index(drop=True)
    sample_df_for_plot[f'cluster_h_{lk}'] = fcluster(Zs[lk], t=CHOSEN_K, criterion='maxclust')
    # sample at most 200 rows to avoid huge overplotting
    plot_df = sample_df_for_plot.sample(n=min(200, len(sample_df_for_plot)), random_state=RANDOM_STATE)
    # prepare df for parallel_coordinates: must contain class column
    plot_pc = plot_df[pc_cols + [f'cluster_h_{lk}']].copy()
    plot_pc[f'cluster_h_{lk}'] = plot_pc[f'cluster_h_{lk}'].astype(str)
    plt.figure(figsize=(10,4))
    parallel_coordinates(plot_pc.reset_index(), class_column=f'cluster_h_{lk}', cols=pc_cols, alpha=0.6)
    plt.title(f"Parallel Coordinates (Hierarchical - {lk})")
    plt.tight_layout()
    plt.savefig(os.path.join(pc_dir, f"parallel_hier_{lk}.png"))
    plt.close()

print("Saved hierarchical cluster centroids, summaries, and parallel coordinate plots.")

# 9. Assign meaningful label to each cluster: we'll provide code to auto-describe clusters by top features
cluster_labels_text = {}
for lk in linkages:
    cent = cluster_summaries[lk]["centroids"]
    desc = {}
    for cl in cent.index:
        # find top 3 features (absolute distance from global mean)
        diffs = (cent.loc[cl] - df[num_cols].mean()).abs().sort_values(ascending=False)
        top3 = diffs.index[:3].tolist()
        desc[cl] = f"Top distinguishing features: {', '.join(top3)}"
    cluster_labels_text[lk] = desc

# ---------- K-MEANS ----------
# 10. Perform k-means clustering. Use MiniBatchKMeans for speed on large data.
mbk = MiniBatchKMeans(n_clusters=CHOSEN_K, random_state=RANDOM_STATE, batch_size=1000, n_init=5, max_iter=200)
k_labels_full = mbk.fit_predict(df_scaled.values)
df['cluster_kmeans'] = k_labels_full + 1  # +1 to match hierarchical cluster numbering

# 11. Parallel coordinates and cluster summary for K-Means
k_centroids_unscaled = pd.DataFrame(scaler.inverse_transform(mbk.cluster_centers_), columns=num_cols)
k_summary = df.groupby('cluster_kmeans')[num_cols].agg(['mean','std','count'])
k_summary.to_csv(os.path.join(OUTPUT_DIR, "kmeans_cluster_summary.csv"))
display(k_summary.head(20))

# Parallel coordinates (sample)
plot_df_k = df.sample(n=min(200, len(df)), random_state=RANDOM_STATE)[pc_cols + ['cluster_kmeans']].copy()
plot_df_k['cluster_kmeans'] = plot_df_k['cluster_kmeans'].astype(str)
plt.figure(figsize=(10,4))
parallel_coordinates(plot_df_k.reset_index(), class_column='cluster_kmeans', cols=pc_cols, alpha=0.6)
plt.title("Parallel Coordinates (K-Means)")
plt.tight_layout()
plt.savefig(os.path.join(pc_dir, "parallel_kmeans.png"))
plt.close()

# 12. Elbow chart (on the SAMPLE to be fast)
if SAMPLE_FOR_HIERARCHY:
    elbow_data = sample_scaled
else:
    elbow_data = df_scaled

inertias = []
ks = list(range(1, ELBOW_K_MAX+1))
for k in ks:
    km = MiniBatchKMeans(n_clusters=k, random_state=RANDOM_STATE, batch_size=500, n_init=5, max_iter=200)
    km.fit(elbow_data.values)
    inertias.append(km.inertia_)
plt.figure(figsize=(6,4))
plt.plot(ks, inertias, marker='o')
plt.xlabel('k'); plt.ylabel('Inertia'); plt.title('Elbow Chart (sample)')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "elbow_chart.png"))
plt.close()

# ---------- METHOD COMPARISON ----------
# 13. Compare K-Means and hierarchical clustering: compute ARI on the sample (map KMeans labels to sample indices)
if SAMPLE_FOR_HIERARCHY:
    k_labels_on_sample = (mbk.predict(sample_scaled.values) + 1)
    comparison = {}
    for lk in linkages:
        hier_labels = fcluster(Zs[lk], t=CHOSEN_K, criterion='maxclust')
        comparison[lk] = adjusted_rand_score(hier_labels, k_labels_on_sample)
else:
    comparison = {}
    for lk in linkages:
        # map hierarchical clusters computed on full data (if you computed them) — here we skip full hierarchical to avoid heavy compute
        comparison[lk] = None

print("ARI (hierarchical vs KMeans) on sample:", comparison)
with open(os.path.join(OUTPUT_DIR, "hier_vs_kmeans_ari.json"), "w") as f:
    json.dump(comparison, f, indent=2)

# 14. Which hierarchical clustering is closest to KMeans?
best_match = None
best_score = -1
for lk, s in comparison.items():
    if s is not None and s > best_score:
        best_score = s
        best_match = lk
print("Best matching hierarchical linkage to kmeans (on sample):", best_match, best_score)

# ---------- MANAGEMENT RECOMMENDATIONS ----------
# 15-16: As a frequent-flyer program manager: decide which clusters to target and what offers to propose.
# We'll generate a small automated recommendation based on typical high-value indicators: 'Balance', 'BonusMiles', 'AvgTicketPrice', 'Flights'
value_indicators = [c for c in num_cols if any(k in c.lower() for k in ['balance','miles','bonus','flight','avg','ticket','fare','points','mileage','segment'])]
if not value_indicators:
    # fallback: pick the numeric column with highest variance
    value_indicators = [df[num_cols].var().sort_values(ascending=False).index[0]]

print("Value-like indicators detected:", value_indicators)

kmeans_profile = df.groupby('cluster_kmeans')[value_indicators].mean()
kmeans_profile.to_csv(os.path.join(OUTPUT_DIR, "kmeans_value_profile.csv"))
# choose top cluster(s) by first indicator
top_cluster = kmeans_profile[value_indicators[0]].idxmax()
recommendation = {
    "target_cluster": int(top_cluster),
    "why": f"Cluster {top_cluster} has highest mean on {value_indicators[0]} indicating high value/engagement.",
    "recommended_offers": [
        "Accelerated miles (bonus miles on next 3 bookings)",
        "Tiered upgrade vouchers or faster status qualification",
        "Personalized route discounts for their frequent routes",
        "Lounge/premium check-in for retention of high-value members"
    ]
}
with open(os.path.join(OUTPUT_DIR, "management_recommendation.json"), "w") as f:
    json.dump(recommendation, f, indent=2)

print("Saved all outputs to", OUTPUT_DIR)
print("Review files in the outputs folder and open saved PNGs/CSVs for the visuals and tables.")


Loaded data: 3999 rows, 12 columns


,count,mean,std,min,25%,50%,75%,max
ID,3999.0,2014.819455,1160.764358,1.0,1010.5,2016.0,3020.5,4021.0
Balance,3999.0,73601.327582,100775.664958,0.0,18527.5,43097.0,92404.0,1704838.0
Qual_miles,3999.0,144.114529,773.663804,0.0,0.0,0.0,0.0,11148.0
cc1_miles,3999.0,2.059515,1.376919,1.0,1.0,1.0,3.0,5.0
cc2_miles,3999.0,1.014504,0.147650,1.0,1.0,1.0,1.0,3.0
cc3_miles,3999.0,1.012253,0.195241,1.0,1.0,1.0,1.0,5.0
Bonus_miles,3999.0,17144.846212,24150.967826,0.0,1250.0,7171.0,23800.5,263685.0
Bonus_trans,3999.0,11.601900,9.603810,0.0,3.0,12.0,17.0,86.0
Flight_miles_12mo,3999.0,460.055764,1400.209171,0.0,0.0,0.0,311.0,30817.0
Flight_trans_12,3999.0,1.373593,3.793172,0.0,0.0,0.0,1.0,53.0


Numeric columns: ['ID', 'Balance', 'Qual_miles', 'cc1_miles', 'cc2_miles', 'cc3_miles', 'Bonus_miles', 'Bonus_trans', 'Flight_miles_12mo', 'Flight_trans_12', 'Days_since_enroll', 'Award']
Saved boxplots to ./outputs/boxplots
Saved scatter pair plots to ./outputs/scatter_pairs (sample size = 2000)
Standard scaling (zero mean, unit variance) is essential for distance-based methods so variables with larger numeric ranges don't dominate distances.
Using sample of 200 for hierarchical clustering/dendrograms.
Saved dendrograms: ./outputs/dendrograms
Saved hierarchical cluster centroids, summaries, and parallel coordinate plots.


ID                           Balance                 \
                       mean          std count           mean            std   
cluster_kmeans                                                                 
1               2448.063939  1070.752256  2346   40793.236999   45293.453236   
2               1790.210000  1124.200549   100  160122.010000  201471.303033   
3               1461.056452  1036.014082   248  271936.794355  225379.490747   
4               1358.422989   968.956512  1305   88259.194636   67362.109527   

                      Qual_miles                    cc1_miles  ...  \
               count        mean          std count      mean  ...   
cluster_kmeans                                                 ...   
1               2346   95.212702   627.026729  2346  1.217391  ...   
2                100  943.120000  1875.881731   100  1.900000  ...   
3                248  486.116935  1347.059177   248  4.157258  ...   
4               1305  105.805364   663.528952  1305  3.186973  ...   

               Flight_miles_12mo Flight_trans_12                  \
                           count            mean       std count   
cluster_kmeans                                                     
1                           2346        0.652174  1.654119  2346   
2                            100       19.280000  8.869514   100   
3                            248        4.000000  4.155870   248   
4                           1305        0.799234  1.667857  1305   

               Days_since_enroll                        Award                  
                            mean          std count      mean       std count  
cluster_kmeans                                                                 
1                    3334.416454  1872.633073  2346  0.177323  0.382023  2346  
2                    4564.260000  1974.505925   100  0.840000  0.368453   100  
3                    5142.225806  1921.058996   248  0.798387  0.402016   248  
4                    5299.524904  1748.669814  1305  0.600000  0.490086  1305  

[4 rows x 36 columns]

ARI (hierarchical vs KMeans) on sample: {'ward': 0.019714462680422557, 'centroid': 0.04913089677581342, 'average': 0.02938709169417507}
Best matching hierarchical linkage to kmeans (on sample): centroid 0.04913089677581342
Value-like indicators detected: ['Balance', 'Qual_miles', 'cc1_miles', 'cc2_miles', 'cc3_miles', 'Bonus_miles', 'Bonus_trans', 'Flight_miles_12mo', 'Flight_trans_12']
Saved all outputs to ./outputs
Review files in the outputs folder and open saved PNGs/CSVs for the visuals and tables.
